### Demonstrate the use of pretrained embedding to classfiy text data with well-described categories

In [2]:
from dotenv import load_dotenv
import openai
import os
from pathlib import Path  # Python 3.6+ only

env_path = Path('../') / 'secrets.env'
load_dotenv(dotenv_path=env_path)
key = os.getenv("AZURE_OPENAI_API_KEY")
openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
openai.api_type = "azure"
openai.api_version = "2023-05-15" 
openai.api_key= key
openai.api_base=openai_endpoint
engine = "gpt-35-turbo"

### Approach 1: using instruction prompt 

Data generation

In [3]:
import json
user_message = ""
def generate_categories():
    user_message =f""" 
    Generate 40 main categories of customer support conversation to classify customer support calls.
    Output data into JSON format .
    Here is the generated data in JSON format:
    
    [{{"category name place_holder":"description place holder"}},{{"category name place holder":"description place holder"}}]
 
"""

    response = openai.ChatCompletion.create(
        engine=engine, # engine = "deployment_name".
        messages=[
            {"role": "system", "content": "You are a customer support data analyst"},
            {"role": "user", "content":user_message },
        ]
    )
    return response['choices'][0]['message']['content']
n=0
categories=""
while (n<5):
    try:

        categories = generate_categories()

        categories = json.loads(categories)
        break
    except:
        n+=1

if len(categories)==0:
    print("data generation fails, please try again")


In [4]:
import ast
user_message = ""
def generate_sub_categories(category):
    user_message =f""" 
    Given this customer support call category \n {category}",\n think of 10-20 subcategories under the category that can further classify the call content.
    Output the data into a list of elements where each element is a tuple of name and description.
    Follow this format template for the output [("name","description"),("name","description")]
    Here are the subcategories for the given category:
    
 
"""

    response = openai.ChatCompletion.create(
        engine=engine, # engine = "deployment_name".
        messages=[
            {"role": "system", "content": "You are a customer support data analyst"},
            {"role": "user", "content":user_message },
        ]
    )
    return response['choices'][0]['message']['content']
subcagories={}
for category in categories:
    
    n=0
    while (n<5):
        try:

            sub_cat = generate_sub_categories(category)
            sub_cat = ast.literal_eval(sub_cat)
            assert type(sub_cat)==list
            assert type(sub_cat[0])==tuple
        
            break
        except:
            n+=1
    subcagories[list(category)[0]]=(list(category.values())[0], sub_cat)


In [5]:
subcagories

{'Billing and payment': ('Assistance with billing inquiries, issues with payments, disputes with charges, setting up automatic payments.',
  [('Account balance inquiries',
    "Assistance with checking the current balance of the customer's account."),
   ('Payment due date inquiries',
    'Assistance with finding out when the next payment is due.'),
   ('Payment plan setup',
    'Helping the customer set up a payment plan for outstanding balances.'),
   ('Credit card authorization failures',
    'Assistance with resolving issues related to failed credit card authorizations.'),
   ('Charges dispute',
    'Helping the customer dispute a charge that they believe is incorrect.'),
   ('Late payment fee inquiries',
    "Assistance with inquiries regarding late payment fees on the customer's account."),
   ('Automatic payment setup and management',
    "Assistance with setting up and managing automatic payments on the customer's account."),
   ('Refund requests',
    "Assistance with processi

In [6]:
import time
def generate_call(description):
    user_message =f""" 
    given {description} as the main topic of an customer support call, generate detail content of the article.
    The content is: 
 
"""
    response = openai.ChatCompletion.create(
        engine="gpt-35-turbo", # engine = "deployment_name".
        messages=[
            {"role": "system", "content": "You are a customer support data analyst"},
            {"role": "user", "content":user_message },
        ]
    )
    return response['choices'][0]['message']['content']

contents =[]
for category in categories:

    for subcategory in subcagories[list(category)[0]][1]:
        description = f"""{list(category)[0]}\n
                        {subcategory[1]}"""
        n=0
        while(n<4):
            try:
                content = (generate_call(description), list(category)[0], subcategory[0])
                break
            except:
                n+=1
                time.sleep(3)
        contents.append(content)

('As a customer support data analyst, one of the main topics we deal with is account management. Account management involves assisting customers with understanding user roles and assigning appropriate permissions to user accounts. This is a crucial aspect of customer support as it ensures that users have access to the features and functionalities they need to effectively use the product or service.\n\nTo begin with, users need to understand the different user roles available and their corresponding permissions. This is important because it allows users to have a clear picture of what they can and cannot do on the platform. Common user roles include:\n \n1. Admin: This is the user with the highest level of access. They can create and edit user accounts, assign permissions, and access all areas of the platform.\n\n2. Manager: This user can create and edit user accounts, assign permissions, and access restricted areas of the platform.\n\n3. Contributor: This user can create and edit conte

In [15]:
import os
folder_path ="data"
os.makedirs(folder_path, exist_ok=True)
file_name = f"documents.txt"
with open(os.path.join(folder_path,file_name), 'w') as file:
    file.write(json.dumps(contents))




In [17]:
loaded_content= json.load(open(os.path.join(folder_path,file_name)))
loaded_content

[["When assisting a customer with checking the current balance of their account, it's important to have all the necessary information at hand. First, ask the customer for their account number or any other identifying information that can help you locate their account. Once you have this information, you can proceed with checking their account balance.\n\nTo check the customer's balance, access their account information on the billing and payment system. You might need to enter their account number or other identification information to gain access. Once you're in, navigate to the account balance section to see the current balance of the account.\n\nIf the account balance is negative, it could mean that the customer has unpaid bills or transactions that have not been processed yet. On the other hand, if the balance is positive, it could mean the customer has paid more than what was required for their current bill or has credit from previous payments.\n\nIt's important to communicate the

In [18]:
loaded_content[0]

["When assisting a customer with checking the current balance of their account, it's important to have all the necessary information at hand. First, ask the customer for their account number or any other identifying information that can help you locate their account. Once you have this information, you can proceed with checking their account balance.\n\nTo check the customer's balance, access their account information on the billing and payment system. You might need to enter their account number or other identification information to gain access. Once you're in, navigate to the account balance section to see the current balance of the account.\n\nIf the account balance is negative, it could mean that the customer has unpaid bills or transactions that have not been processed yet. On the other hand, if the balance is positive, it could mean the customer has paid more than what was required for their current bill or has credit from previous payments.\n\nIt's important to communicate the 

In [1]:
import pandas as pd
import re
import openai
import re
import requests
import pandas as pd
from openai.embeddings_utils import get_embedding, cosine_similarity
from transformers import GPT2TokenizerFast
import tiktoken

API_KEY = "93bf99a031474525b0bfc66eafca7df4"
RESOURCE_ENDPOINT = "https://azopenaidemo.openai.azure.com/" 

openai.api_type = "azure"
openai.api_key = API_KEY
openai.api_base = RESOURCE_ENDPOINT
openai.api_version = "2022-12-01"

url = openai.api_base + "/openai/deployments?api-version=2022-12-01"

r = requests.get(url, headers={"api-key": API_KEY})
print(r)



<Response [200]>


In [2]:
# Normalize text data util function
def normalize_text(s, sep_token = " \n "):
    s = re.sub(r'\s+',  ' ', s).strip()
    s = re.sub(r". ,","",s)
    # remove all instances of multiple spaces
    s = s.replace("..",".")
    s = s.replace(". .",".")
    s = s.replace("\n", "")
    s = s.strip()
    
    return s



## Data Loading

In [3]:

df = pd.read_parquet("../../data/final_df.parquet")
df.head()

,Consumer complaint narrative,Issue,n_tokens,embedding,prediction
1665,I know at this point I do n't know what can be...,Dealing with my lender or servicer,842,"[-0.01611567847430706, -0.024448800832033157, ...",Can't repay my loan
422,My Name is XXXX. While I attended school at th...,Can't repay my loan,713,"[-0.009162590838968754, -0.006951234769076109,...",Can't repay my loan
4771,I have had a Macy 's credit account for years ...,Delinquent account,622,"[0.0075323572382330894, 0.0036780808586627245,...",Unable to get credit report/credit score
1046,Incorporatess by reference all preceding parag...,"Application, originator, mortgage broker",808,"[0.004820031579583883, -0.010541003197431564, ...","Loan modification,collection,foreclosure"
14628,A buyer has purchased a watch from me online a...,Transaction issue,523,"[0.005065633915364742, -0.03737545758485794, 0...",Transaction issue


In [4]:
#apply text normalization
df['Consumer complaint narrative']= df["Consumer complaint narrative"].apply(lambda x : normalize_text(x))




In [5]:
#optional, use this to filter/cut long text (longer than 8192 tokens)
tokenizer = tiktoken.get_encoding("cl100k_base")
df['n_tokens'] = df["Consumer complaint narrative"].apply(lambda x: len(tokenizer.encode(x)))
df = df[df.n_tokens<8192]
len(df)

1000

In [6]:
df['embedding'] = df["Consumer complaint narrative"].apply(lambda x : get_embedding(x, engine = 'text-embedding-ada-002'))


#### Prepare the category description : embedding map

In [7]:
#get the category descriptions (unlike normal classification, the more detail, more clear, more distinctive description the better)
# produce the embedding for each category description 
import numpy as np
issues = np.unique(df.Issue)
issue_emb = {issue:get_embedding(issue, engine = 'text-embedding-ada-002')for issue in issues}

In [8]:
issue_emb['APR or interest rate']

[-0.011190177872776985,
 -0.018179483711719513,
 0.05631539225578308,
 0.0070075299590826035,
 -0.013513871468603611,
 -0.005822902079671621,
 0.0226901825517416,
 0.038345497101545334,
 0.023528534919023514,
 -0.013522983528673649,
 -0.027829645201563835,
 0.0005404865369200706,
 0.017969895154237747,
 0.007463156245648861,
 -0.00269502867013216,
 -0.0046063801273703575,
 0.005708995740860701,
 -0.017705632373690605,
 0.0010855293367058039,
 0.009586374275386333,
 -0.023419184610247612,
 -0.015017437748610973,
 -0.027756744995713234,
 0.024603812023997307,
 0.011682254262268543,
 0.00034342819708399475,
 -0.011472666636109352,
 0.005722664296627045,
 0.0008679677848704159,
 0.011199290864169598,
 0.013905709609389305,
 -0.022671956568956375,
 0.008319733664393425,
 0.026772592216730118,
 -0.013604996725916862,
 0.018316170200705528,
 -0.015846677124500275,
 0.019774174317717552,
 -0.0024262091610580683,
 -0.033461183309555054,
 0.03162045404314995,
 0.004212263505905867,
 -0.027045967

In [9]:
import numpy as np  
  
import numpy as np  
  
def categorize(input_vector):  
    """  
    Given an input vector and a dictionary of label vectors,  
    returns the label with the highest cosine similarity to the input vector.  
    """  
    max_similarity = float('-inf')  
    max_label = None  
      
    # Compute cosine similarity between input vector and each label vector  
    for label, vector in issue_emb.items():  
        cosine_similarity = np.dot(input_vector, vector) / (np.linalg.norm(input_vector) * np.linalg.norm(vector))  
          
        # Update max_similarity and max_label if necessary  
        if cosine_similarity > max_similarity:  
            max_similarity = cosine_similarity  
            max_label = label  
      
    return max_label  


### Prediction or matching

In [ ]:
df["prediction"] = df["embedding"].apply(categorize)

In [ ]:
# Count accuracy. 
df[df["prediction"] ==df["Issue"]].count()/df.shape[0]

In [ ]:
df[['Consumer complaint narrative','Issue','prediction']].head()

### Approach using ChatGPT and prompt

In [ ]:
#Note: The openai-python library support for Azure OpenAI is in preview.
import os
import openai
import time
openai.api_version = "2023-03-15-preview"
user_message = ""
def classify(text):
    user_message =f""" 
 classify the following input into one of the following category {issues}
 <<input>>
 {text}
 <<input>>
 The category of the input is:
"""
    i=0
    while i<10:

        try:
            response = openai.ChatCompletion.create(
                engine="gpt-35-turbo", # engine = "deployment_name".
                messages=[
                    {"role": "system", "content": "You are a helpful assistant."},
                    {"role": "user", "content":user_message },
                ]
            )
            return response['choices'][0]['message']['content']
        except:
            time.sleep(3)
            i+=1
            


In [ ]:
complaints = df['Consumer complaint narrative'].values
chatgpt_predictions =[]
for complaint in complaints:
    result = classify(complaint)
    chatgpt_predictions.append(result)

In [ ]:
df['chatgpt_predictions'] =chatgpt_predictions

In [ ]:
df[df["chatgpt_predictions"] ==df["Issue"]].count()/df.shape[0]